In [10]:
%matplotlib notebook
%matplotlib inline

# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import time
import gmaps

# Google developer API key 
from config import gkey
gmaps.configure(api_key=gkey)


In [11]:
#Import csv from WeatherPy assignment
df2 = pd.read_csv('../WeatherPy/WeatherPY_Data.csv')

In [30]:
df2.shape
#df2.head()

(587, 10)

In [15]:
#Create a heat map that displays the humidity for every city from the part I of the homework.

# Store 'Lat' and 'Lng' into  locations 
locations = df2[['act_lat', 'act_lng']].astype(float)

# Convert Humidity to float and store
humidity = df2['Humidity_Perc'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [78]:
#Create dataframe with perferred weather conditions
df3 = df2[(df2.Temperature_F >= 70) & (df2.Temperature_F < 80) & (df2.Humidity_Perc <= 61)].copy()


In [89]:
# find first hotel in each preferred city (df2) and add to dataframe
df3['Hotel_Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'rankby': 'distance',
    'type': 'lodging',
    'key': gkey,
}

for index, row in df3.iterrows():

    # add location to params dict
    lat = row['act_lat']
    lng = row['act_lng']
    coords = (f'{lat},{lng}')
    params['location'] = coords
    
    #vars for output reporting
    city  = row['Nearest_City']

    # build response object
    response = requests.get(base_url, params=params)
    response_json = response.json()
    response_time = response.elapsed.total_seconds()

    try:
       df3.loc[index, ['Hotel_Name']] = response_json['results'][0]['name'] #hotel name
       response_time = response.elapsed.total_seconds()
       print(f'hotel response for {city} in {response_time} seconds')

      
    except (KeyError, IndexError):
        status = response_json['status']
        print(f'Response error for {city} skipping. status = {status}')
        

    time.sleep(response.elapsed.total_seconds())


hotel response for hilo in 0.413013 seconds
hotel response for busselton in 0.229238 seconds
hotel response for northam in 0.493792 seconds
hotel response for busselton in 0.26252 seconds
hotel response for busselton in 0.24478 seconds
hotel response for pisco in 0.355018 seconds
hotel response for northam in 0.314996 seconds
hotel response for busselton in 0.215119 seconds
Response error for tessalit skipping. status = ZERO_RESULTS
hotel response for busselton in 0.244935 seconds
hotel response for pisco in 0.23436 seconds
hotel response for rawson in 0.325636 seconds
hotel response for busselton in 0.325235 seconds
Response error for san luis skipping. status = ZERO_RESULTS
hotel response for busselton in 0.212795 seconds
hotel response for busselton in 0.333376 seconds
hotel response for busselton in 0.23197 seconds
hotel response for tacuarembo in 0.304099 seconds
hotel response for tuatapere in 0.193988 seconds
hotel response for busselton in 0.232855 seconds
hotel response for bu

In [109]:
#Create Symbol GMAP for hotels in citys with perfered weather df3

preferred_locations = df3[['act_lat', 'act_lng']].astype(float)
hotel_names = df3['Hotel_Name'].tolist()
humidity = df3['Humidity_Perc'].astype(float)

fig = gmaps.figure()

marker_layer = gmaps.marker_layer(preferred_locations, display_info_box = True, info_box_content = hotel_names)

heat_layer = gmaps.heatmap_layer(preferred_locations, weights=humidity)

fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))